In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import colors
from PIL import Image
import napari
import skimage.io as io
import seaborn as sns
import pandas as pd
import colorcet as cc
import sys
import tifffile
from scipy import ndimage
from skimage.segmentation import find_boundaries

sys.path.append('..')
from spatialMI_functions import plot_layers_napari, crop_black_margins, plot_pixie_maps

import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Explore some different color options

## Lunaphore images
channel_order = ["DAPI",
                 "TNNT2","ANKRD1","PDGFRa","CD31","aSMA",
                 "CD45","CCR2","TREM2","CD68","MPO"]
colorblind = sns.color_palette("colorblind")
glasbey = sns.color_palette(cc.glasbey)
comet_images_color = ['cyan',
                      glasbey[10],glasbey[7],glasbey[6],colorblind[3],glasbey[0],
                      glasbey[1],glasbey[2],glasbey[4],glasbey[12],"magenta"]

## Pixel maps
pixel_maps_color = sns.color_palette(cc.glasbey)

## Cell clusters
cell_cluster_color = sns.color_palette(cc.glasbey)

# Visualize the colors in the color palette
pixel_maps_color

In [ ]:
# Assign each individual label in new_mask the corresponding color value from the color table
pixel_dir = '../../../results/seqIF/pixie_pixel_masks_0.05'

## Get representative pixel categories from one sample
sample_folder = pixel_dir + "/" + "Control_14" + "/"
pixel_categories = pd.read_csv(sample_folder + "population_pixel_mask.csv")

## Assign color palettes to the different pixel categories
## Pixel maps
pixel_maps_color =sns.color_palette(cc.glasbey) 
color_palette_use = pixel_maps_color[1:pixel_categories.shape[0]+1]
color_dict = {0: 'white'}
for i,region_id in enumerate(pixel_categories['region_id']):
    if pixel_categories['region_name'][i] == "background":
        color_dict[region_id] = "white"
    else:
        color_dict[region_id] = color_palette_use[i]
color_dict

In [ ]:
# Translate color_dict color values from rgb to hex
color_dict_hex = {}
for key in color_dict:
    color_dict_hex[key] = colors.to_hex(color_dict[key])
color_dict_hex

In [ ]:
# Assign each individual label in new_mask the corresponding color value from the color table
pixel_dir = '../../../results/seqIF/pixie_pixel_masks_0.05'
outdir = "../../output/seqIF/pixie_maps/"
# Generate an image of the pixel map from pixie for each sample. These will be put together in Illustrator afterward
for sample in os.listdir(pixel_dir):
    if os.path.isdir(os.path.join(pixel_dir, sample)):
        print(sample)
        sample_dir= pixel_dir + "/" + sample + "/"
        pixel_img = tifffile.TiffFile(sample_dir + "population_pixel_mask.tiff").asarray()

        # Read in a csv file with the ROI coordinates
        sample_roi = pd.read_csv("/Users/florian_wuennemann/1_Projects/MI_project/mi_spatialomics/annotations/SeqIF/heart_region_rois/"+ sample +"_heart.csv")

        # use the bounding box positions in the roi to subset the pixel_img and the mask
        y_min = int(sample_roi['axis-0'].min())
        y_max = int(sample_roi['axis-0'].max())
        x_min = int(sample_roi['axis-1'].min())
        x_max = int(sample_roi['axis-1'].max())

        # crop the pixel_img and the mask
        pixel_img_cropped = pixel_img[y_min:y_max, x_min:x_max]

        viewer = napari.Viewer()
        viewer.add_labels(pixel_img_cropped, visible=True, name=sample, opacity = 1, color = color_dict)
        full_screenshot = outdir+sample+".pixel_map.full_image.wborder.png"
        viewer.screenshot(path=full_screenshot, scale=3)
        crop_screenshot = outdir+sample+".pixel_map.full_image.clean.png"
        crop_black_margins(full_screenshot, crop_screenshot)

In [ ]:
## Manual exploration of pixel maps in Napari
viewer = napari.Viewer()
for folder in os.listdir(pixel_dir):
    if os.path.isdir(os.path.join(pixel_dir, folder)) and folder != "napari_screenshots":
        print(folder)
        # Do something with the folder here
        sample_folder = pixel_dir + "/" + folder + "/"
        pixel_img = tifffile.TiffFile(sample_folder + "population_pixel_mask.tiff").asarray()
        viewer.add_labels(pixel_img, visible=False, name=folder, opacity = 1, color = color_dict)

In [ ]:
# Test code for a single sample
sample = "4h_96"
folder = sample

sample_folder = pixel_dir + "/" + folder + "/"
pixel_img = tifffile.TiffFile(sample_folder + "population_pixel_mask.tiff").asarray()


# Read in a csv file with the ROI coordinates
sample_roi = pd.read_csv("/Users/florian_wuennemann/1_Projects/MI_project/mi_spatialomics/annotations/SeqIF/heart_region_rois/"+ sample +"_heart.csv")

# use the bounding box positions in the roi to subset the pixel_img and the mask
x_min = int(sample_roi['axis-0'].min())
x_max = int(sample_roi['axis-0'].max())
y_min = int(sample_roi['axis-1'].min())
y_max = int(sample_roi['axis-1'].max())

# crop the pixel_img and the mask
pixel_img_cropped = pixel_img[y_min:y_max, x_min:x_max]


viewer = napari.Viewer()
viewer.add_labels(pixel_img_cropped, visible=True, name=folder, opacity = 1, color = color_dict)

full_screenshot = outdir+sample+".pixel_map.full_image.wborder.png"
viewer.screenshot(path=full_screenshot, scale=3)

crop_screenshot = outdir+sample+".pixel_map.full_image.clean.png"
crop_black_margins(full_screenshot, crop_screenshot)